In [3]:
pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import json
import csv
import datetime
from geopy.distance import distance

# Three Functions Created

In [5]:
def list_airbnb(location_geocode, max_distance):
    client = MongoClient('localhost',27017) ## or MongoClient("localhost:27017")
    db = client.apan5400
    collection = db.airbnb_reviews
    
    filtered_listings = []
    for listing in collection.find():
        listing_latitude = listing['location']['latitude']
        listing_longitude = listing['location']['longitude']
        listing_geocode = (listing_latitude, listing_longitude)
        listing_distance = distance(location_geocode, listing_geocode).miles
        if listing_distance <= max_distance:
            filtered_listings.append(listing)
    filtered_id = []
    for x in range(len(filtered_listings)):
        filtered_id.append(filtered_listings[x]['listing_id'])
        
    client.close()
    
    return filtered_id

In [6]:
def sql_airbnb(filtered_id):
    # Connect to the database
    conn = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="5400_airbnb",
    user="postgres",
    password="1234")
    
    # create a cursor
    cur = conn.cursor()
    
    # Define the SQL query
    query_listing_info = """
        SELECT
            id,
            name,
            property_type,
            room_type,
            accommodates,
            bedrooms,
            bathrooms,
            price
        FROM 
            listings
        WHERE 
            id IN %(id)s
    """
    
    # Execute the SQL query with the filtered_id parameter
    cur.execute(query_listing_info, {'id': tuple(filtered_id)})
    
    # Fetch the results and format them as a list of dictionaries
    listing_info = []
    for row in cur:
        result = {
            'listing_id': row[0],
            'name': row[1],
            'property_type': row[2],
            'room_type': row[3],
            'accommodates': row[4],
            'bedrooms': row[5],
            'bathrooms': row[6],
            'price': row[7]
        }
        listing_info.append(result)
    
    # Close the cursor and database connection
    cur.close()
    conn.close()
    
    # Return the listing information as a list of dictionaries
    return listing_info

In [7]:
def mongo_airbnb(listing_id):
    client = MongoClient('localhost',27017) ## or MongoClient("localhost:27017")
    db = client.apan5400
    collection = db.airbnb_reviews
    
    
    reviews = []
    query = {"listing_id": listing_id}
    result = collection.find(query)
    for review in result:
        reviews.append(review)
            
    client.close()
    
    return reviews

### Create Connections to MongoDB

In [8]:
#Set up the MongoDB Client to get read for importation
import pymongo
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27017")
db = client.apan5400
collection = db.airbnb_reviews

In [9]:
total_docs = collection.count_documents({})
total_docs

97704

In [10]:
client.close()

### Create Connections to PostgreSQL

In [13]:
import psycopg, os

print('Connecting to the PostgreSQL database...')
conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="5400_airbnb",
    user="postgres",
    password="123")

Connecting to the PostgreSQL database...


In [14]:
# create a cursor
cur = conn.cursor()

In [15]:
# execute a statement
print('PostgreSQL database version:')
cur.execute('SELECT version()')

# display the PostgreSQL database server version
db_version = cur.fetchone()
print(db_version)

PostgreSQL database version:
('PostgreSQL 15.1 (Debian 15.1-1.pgdg110+1) on aarch64-unknown-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)


In [16]:
# close the communication with the PostgreSQL
cur.close()

### API to obtain geocode for addresses

In [17]:
import requests

In [20]:
def geocode_address(address):
    # set the API endpoint and parameters
    endpoint = "https://nominatim.openstreetmap.org/search"
    params = {"q": address, "format": "json"}

    # send the request to the API
    response = requests.get(endpoint, params=params)

    # parse the JSON response and extract the latitude and longitude
    if response.status_code == 200:
        data = response.json()
        if data:
            lat = data[0]["lat"]
            lon = data[0]["lon"]
            return lat, lon
        else:
            print("Error: No results found.")
    else:
        print("Error: ", response.status_code)

In [21]:
# prompt the user to enter an address
address = input("Enter an address: ")

# geocode the address and print the result
result = geocode_address(address)
if result:
    lat, lon = result
    print(f"Latitude: {lat}, Longitude: {lon}")

Enter an address: columbia university
Latitude: 40.80775585, Longitude: -73.96164946987652


In [22]:
location_geocode = (result[0], result[1])